# Pre-amble

In [1]:
# loading the required packages
import pandas as pd
import numpy as np # Data load and basic dataset informationumpy as np
import datetime
import random
import re
import os
from dateutil import relativedelta
import math
#import plotnine
#from plotnine import *

# for repeated printouts 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to custom displays of row-column df printouts
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Data load and basic dataset information

We first load the intake data:

In [2]:
intake_raw = pd.read_csv('../Data/intake.csv')

In [3]:
# printing the data shape 
print("The shape of the `intake_raw` dataset: " + str(intake_raw.shape))

# printing the data head
intake_raw.head(n=10)

# printing the data types
print("Data types of the `intake_raw` dataset column: " + str(intake_raw.dtypes))

The shape of the `intake_raw` dataset: (467698, 17)


,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PARTICIPANT_STATUS,AGE_AT_INCIDENT,RACE,GENDER,INCIDENT_CITY,INCIDENT_BEGIN_DATE,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,UPDATE_OFFENSE_CATEGORY
0,288191106116,102402970877,05/24/2011,PROMIS Conversion,Continued Investigation,25.0,Black,Male,NaN,03/12/2007,NaN,PROMIS Data Conversion,NaN,04/03/2007 12:00:00 AM,05/24/2011,Approved,PROMIS Conversion
1,288191106116,102403017038,05/24/2011,PROMIS Conversion,Approved,22.0,Black,Male,NaN,03/12/2007,NaN,PROMIS Data Conversion,NaN,05/22/2011 06:51:00 PM,05/24/2011,Approved,PROMIS Conversion
2,288191106116,102403063200,05/24/2011,PROMIS Conversion,Continued Investigation,19.0,Black,Male,NaN,03/12/2007,NaN,PROMIS Data Conversion,NaN,NaN,05/24/2011,Approved,PROMIS Conversion
3,295479479935,107142346781,01/27/2012,PROMIS Conversion,Approved,NaN,Black,Male,NaN,06/25/2009,NaN,CPD SPEC INV-YOUTH,NaN,07/14/2009 02:34:00 PM,07/14/2009,Charge(S) Approved,PROMIS Conversion
4,313304913049,107031421288,01/31/2011,PROMIS Conversion,Approved,37.0,HISPANIC,Male,NaN,04/07/2010,NaN,SPEC. OPERATIONS,NaN,04/07/2010 08:40:00 PM,NaN,NaN,PROMIS Conversion
5,313304913049,340245967874,01/31/2011,PROMIS Conversion,Approved,29.0,HISPANIC,Male,NaN,04/07/2010,NaN,SPEC. OPERATIONS,NaN,04/07/2010 08:44:00 PM,NaN,NaN,PROMIS Conversion
6,317478798047,107243162958,04/15/2011,PROMIS Conversion,Approved,24.0,Black,Female,NaN,02/06/2010,NaN,C P D AREA 5 DIST 25,NaN,02/06/2010 12:00:00 AM,02/06/2010,Charge(S) Approved,PROMIS Conversion
7,505746802102,347300376805,01/31/2011,UUW - Unlawful Use of Weapon,Approved,34.0,Black,Male,Chicago,12/15/2010,NaN,CHICAGO PD,NaN,12/15/2010 08:40:00 PM,12/16/2010,Approved,Narcotics
8,505746802102,347347276638,01/31/2011,UUW - Unlawful Use of Weapon,Approved,27.0,Black,Male,Chicago,12/15/2010,NaN,CHICAGO PD,NaN,12/15/2010 08:40:00 PM,12/16/2010,Approved,Narcotics
9,505761173380,347314548310,07/17/2011,Homicide,Approved,41.0,White [Hispanic or Latino],Male,Chicago,07/17/2011,NaN,CHICAGO PD,NaN,07/17/2011 07:05:00 PM,07/17/2011,Felony Review,Homicide


Data types of the `intake_raw` dataset column: CASE_ID                      int64
CASE_PARTICIPANT_ID          int64
RECEIVED_DATE               object
OFFENSE_CATEGORY            object
PARTICIPANT_STATUS          object
AGE_AT_INCIDENT            float64
RACE                        object
GENDER                      object
INCIDENT_CITY               object
INCIDENT_BEGIN_DATE         object
INCIDENT_END_DATE           object
LAW_ENFORCEMENT_AGENCY      object
LAW_ENFORCEMENT_UNIT        object
ARREST_DATE                 object
FELONY_REVIEW_DATE          object
FELONY_REVIEW_RESULT        object
UPDATE_OFFENSE_CATEGORY     object
dtype: object


Key row identifiers in the dataset: `CASE_ID`, `CASE_PARTICIPANT_ID`

In [4]:
# How many CASE_ID?
print("Number of unique CASE_ID: " + str(len(pd.unique(intake_raw.CASE_ID)))) 

# how many CASE_PARTICIPANT_ID? 
print("Number of unique CASE_PARTICIPANT_ID: " + str(len(pd.unique(intake_raw.CASE_PARTICIPANT_ID)))) 

Number of unique CASE_ID: 423231
Number of unique CASE_PARTICIPANT_ID: 467698


Each row in the `intake_raw` dataset represents unique `CASE_PARTICIPANT_ID`. 

# Data cleaning

In [5]:
# creating a copy of the raw data 
intake_cleaned = intake_raw.copy()

## Basic demographics

In [6]:
# cleaning the RACE variable
print("Distribution of original `RACE` variable:")
intake_cleaned.RACE.value_counts()

# defining some important race groups
intake_cleaned['is_black'] = np.where(intake_cleaned.RACE.isin(['Black', 'White/Black [Hispanic or Latino]']), 
                                      True, False)

intake_cleaned['is_hispanic'] = np.where(intake_cleaned.RACE.isin(['HISPANIC', 'White [Hispanic or Latino]']), 
                                         True, False)

intake_cleaned['is_white'] = np.where(intake_cleaned.RACE.isin(['White', 'CAUCASIAN']), 
                                      True, False)

# for the RACE columns, replace value with np.nan if RACE == 'Unknown' or RACE == 'Biracial' or Albino
cond = intake_cleaned.RACE.isin(['Unknown', 'Biracial', 'Albino']) # defining the condition
intake_cleaned.loc[cond, ['is_black', 'is_hisp', 'is_white']] = np.nan

Distribution of original `RACE` variable:


Black                               301419
White [Hispanic or Latino]           67077
White                                65671
HISPANIC                             11535
Asian                                 3176
White/Black [Hispanic or Latino]      2308
Unknown                                944
American Indian                        190
Biracial                                96
ASIAN                                    8
Albino                                   1
CAUCASIAN                                1
Name: RACE, dtype: int64

In [7]:
# original GENDER distribution:
print("Distribution of original `GENDER` distribution:")
intake_cleaned.GENDER.value_counts()

# defining gender groups 
intake_cleaned['is_female'] = np.where(intake_cleaned.GENDER.isin(['Male', 'Male name, no gender given']), 
                                       False, np.where(intake_cleaned.GENDER.str.contains('Unknown'), 
                                                       "", True))

# final look at the gender variable 
print("Distribution of cleaned `is_female` distribution:")
intake_cleaned.is_female.value_counts()

Distribution of original `GENDER` distribution:


Male                          394051
Female                         60708
Unknown                           23
Unknown Gender                    21
Male name, no gender given        19
Name: GENDER, dtype: int64

Distribution of cleaned `is_female` distribution:


False    394070
True      60708
          12920
Name: is_female, dtype: int64

In [8]:
# original summary stat of age variable:
print("Summary statistics of original AGE_AT_INCIDENT variable:")
intake_cleaned.AGE_AT_INCIDENT.describe()

# there is an outlier (215 y.o. obs), winsorizing age column to 99.995th percentile
intake_cleaned['age_cleaned'] = np.where(intake_cleaned.AGE_AT_INCIDENT >= 
                                         intake_cleaned.AGE_AT_INCIDENT.quantile(0.9995), 
                                         intake_cleaned.AGE_AT_INCIDENT.quantile(0.9995), 
                                         intake_cleaned.AGE_AT_INCIDENT)

# printing the summary stat of new age variable
print("Summary statistics of cleaned age variable:")
intake_cleaned.age_cleaned.describe()

Summary statistics of original AGE_AT_INCIDENT variable:


count    453727.000000
mean         33.774704
std          12.407875
min          17.000000
25%          24.000000
50%          31.000000
75%          43.000000
max         215.000000
Name: AGE_AT_INCIDENT, dtype: float64

Summary statistics of cleaned age variable:


count    453727.000000
mean         33.761936
std          12.339959
min          17.000000
25%          24.000000
50%          31.000000
75%          43.000000
max          81.000000
Name: age_cleaned, dtype: float64

<u>**Cleaning flags (if any)**</u>:

1. `RACE`: 
    - How should we categorize *biracial* race group?
    - I recoded `Unknown`, `Biracial`, `Albino` as NaN for each race definition
    - What does `[Hispanic or Latino]` actually mean? In pset2, why did we not categorize `White/Black [Hispanic or Latino]` into the `is_hisp` definition?
    
    
2. `GENDER`:
    - I recoded rows containing `Unknown` as NaN
    - `Male name, no gender given` is coded as `Male` (reasonable?).
    

3. `AGE_AT_INCIDENT`:
    - As with pset2, I winsorized the age variable to 99.995th percentile

## Cleaning datetime variables<a class="anchor" id="datetime-variables"></a>

Here, we:

* Clean up the defendant's felony review date (`FELONY_REVIEW_DATE`). We'll create a datetime object out of the field, and will separate year, month, and year-month component of the date.

* Add several key treatment variables that will be of interest for the analysis

* Add time relative indicators for the event study estimation

In [9]:
# we clean up the SENTENCE_DATE that have out-of-bound years
intake_cleaned['felony_review_date'] = [re.sub(r'2[1-9]([0-9]+)', r'20\1', str(date)) 
                                       if bool(re.search(r'2[1-9]([0-9]+)', str(date)))
                                       else str(date) 
                                       for date in intake_cleaned.FELONY_REVIEW_DATE]

# converting the FELONY_REVIEW_DATE to datetime
intake_cleaned['felony_review_date'] = pd.to_datetime(intake_cleaned["felony_review_date"])

# creating a year, month, date columns 
intake_cleaned['felony_review_year'] = pd.DatetimeIndex(intake_cleaned['felony_review_date']).year
intake_cleaned['felony_review_month'] = pd.DatetimeIndex(intake_cleaned['felony_review_date']).month
intake_cleaned['felony_review_day'] = pd.DatetimeIndex(intake_cleaned['felony_review_date']).day
intake_cleaned['felony_review_ym'] = intake_cleaned['felony_review_date'].dt.to_period('M')

In [10]:
# defining the treatment variables
intake_cleaned['sa_office_period'] = np.where(intake_cleaned.felony_review_ym >= "2016-12", # SA Foxx assumed office in Dec 1, 2016
                                              True, False)   

intake_cleaned['sa_timedelta'] = (intake_cleaned.felony_review_year - 2016)*12 + \
                                 (intake_cleaned.felony_review_month - 12)

intake_cleaned['sa_timedelta_days'] = (intake_cleaned['felony_review_date'] - pd.to_datetime("2016-12-01")).dt.days

intake_cleaned['sa_timedelta_wk'] = [math.floor(delta_days/7) if delta_days >= 0       # 2.14 weeks as 2 weeks
                                     else math.ceil(delta_days/7) if delta_days < 0    # -3.14 weeks as -3 weeks
                                     else np.nan
                                     for delta_days in intake_cleaned.sa_timedelta_days]

intake_cleaned['BRA_period'] = np.where(intake_cleaned.felony_review_ym >= "2017-06", # Bail Reform Act
                                        True, False)

intake_cleaned['BRA_timedelta'] = (intake_cleaned.felony_review_year - 2017)*12 + \
                                  (intake_cleaned.felony_review_month - 6)

intake_cleaned['BRA_timedelta_days'] = (intake_cleaned['felony_review_date'] - pd.to_datetime("2017-06-12")).dt.days

intake_cleaned['BRA_timedelta_wk'] = [math.floor(delta_days/7) if delta_days >= 0       # 2.14 weeks as 2 weeks
                                      else math.ceil(delta_days/7) if delta_days < 0    # -3.14 weeks as -3 weeks
                                      else np.nan
                                      for delta_days in intake_cleaned.BRA_timedelta_days]

<u>**Cleaning flags (if any)**</u>:

1. `SENTENCE_DATE`: 
    - All out-of-bounds years (29, 22, etc instead of 20XX) are converted into 20XX. 
    - How do we clean up 2023-2024 though? (Currently, we're filtering against values > 2022 in the filtering stage)
    

2. Key treatment-related time variables:
    - SA Kim Foxx entry: Value = 1 if December 2016 onwards, 0 if otherwise
    - Bail reform act: Value = 1 if June 2017 onwards, 0 if otherwise
    - For both indicators, relative time indicators are defined relative to the timing that corresponds to the start of value = 1

## Cleaning felony review-related variables<a class="anchor" id="felony-vars"></a>

Here, we define a broad category for felony review results:

In [11]:
# defining three broad categories for felony review result:
intake_cleaned['fr_is_approved'] = np.where(intake_cleaned.FELONY_REVIEW_RESULT.isin(['Approved', 
                                                                                      'Charge(S) Approved']), 
                                            True, False)
intake_cleaned['fr_is_rejected'] = np.where(intake_cleaned.FELONY_REVIEW_RESULT.isin(['Rejected', 
                                                                                      'Disregard']), 
                                            True, False)## Cleaning felony review-related variables<a class="anchor" id="felony-vars"></a>
intake_cleaned['fr_is_investigated'] = np.where(intake_cleaned.FELONY_REVIEW_RESULT == 'Continued Investigation', 
                                                True, False)

# Exporting the dataset<a class="anchor" id="data-export"></a>

We then export the final `intake_cleaned` dataset into csv format, to be imported in the exploratory analysis notebook:

In [12]:
intake_cleaned.to_csv('../Data/intake_analysis.csv')